In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

sber_data = pd.read_csv('data_2/sber_data.csv', sep=',')
#display(sber_data.head())
#Чему равно число строк в таблице?
#display(sber_data.tail())

#Сколько районов Москвы и Московской области представлено в данных?
#display(sber_data['sub_area'].value_counts())

#Чему равна максимальная цена квартир (price_doc)? Введите это число полностью, без округлений.
#display(sber_data['price_doc'].max())


#Проверим, влияет ли уровень экологической обстановки в районе на цену квартиры.
# Постройте коробчатую диаграмму цен на квартиры (price_doc) в зависимости от уровня экологической обстановки в районе (ecology).
# Какой уровень ценится на рынке меньше всего?
#Строим график распределения возраста в зависимости от лояльности
fig = px.box(
    sber_data,
    x='price_doc',
    y='ecology',
    color='ecology',
    orientation = 'h',
    labels={'price_doc':'Цены на квартиры', 'ecology':'Уровень экологической обстановки'},
    height = 800, 
    width=1200,
    title = "Распределение цен на квартиры взависимости от уровеня экологической обстановки",
)
#fig.show()


#Постройте диаграмму рассеяния, которая покажет, как цена на квартиру (price_doc) связана с расстоянием до центра Москвы (kremlin_km). 
#Выберите все верные утверждения.
fig = px.scatter(
    sber_data,
    x='kremlin_km',
    y='price_doc',
    labels={'price_doc':'Цены на квартиры, млн.', 'kremlin_km':'Hасстоянием до центра Москвы, км.'},
    height = 800, 
    width=1200,
    title = "Lиаграмму рассеяния зависимости цены на квартиру и расстояния до центра Москвы",
)
#fig.show()



#В библиотеке pandas специально для этого реализован метод isnull().
# Этот метод возвращает новый DataFrame, в ячейках которого стоят булевы значения True и False.
# True ставится на месте, где ранее находилось значение NaN.
#display(sber_data.isnull().tail())


#Первый способ — это вывести на экран названия столбцов, где число пропусков больше 0/
#Для этого вычислим средний по столбцам результат метода isnull(). Получим долю пропусков в каждом столбце.
#Умножаем на 100 %, находим столбцы, где доля пропусков больше 0, сортируем по убыванию и выводим результат:
cols_null_percent = sber_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
#display(cols_with_null)


#Можно воспользоваться столбчатой диаграммой, чтобы визуально оценить соотношение числа пропусков к числу записей.
# Самый быстрый способ построить её — использовать метод plot():
#cols_with_null.plot(
#    kind='bar',
#    figsize=(10, 4),
#    title='Распределение пропусков в данных'
#);



#Для создания такой тепловой карты можно воспользоваться результатом метода isnull().
# Ячейки таблицы, в которых есть пропуск, будем отмечать жёлтым цветом, а остальные — синим.
# Для этого создадим собственную палитру цветов тепловой карты с помощью метода color_pallete() из библиотеки seaborn.
#colors = ['blue', 'yellow'] 
#fig = plt.figure(figsize=(10, 4))
#cols = cols_with_null.index
#ax = sns.heatmap(
#    sber_data[cols].isnull(),
#    cmap=sns.color_palette(colors),
#)


#редварительно создадим копию исходной таблицы — drop_data, чтобы не повредить её.
#Зададимся порогом в 70 %: будем оставлять только те столбцы, в которых 70 и более процентов записей не являются пустыми.
#После этого удалим записи, в которых содержится хотя бы один пропуск.
#Наконец, выведем информацию о числе пропусков и наслаждаемся нулями. 
#создаем копию исходной таблицы
drop_data = sber_data.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
drop_data = drop_data.dropna(how='any', axis=0)
#отображаем результирующую долю пропусков
drop_data.isnull().mean()

#print(drop_data.shape)



#Вся сложность заключается в выборе метода заполнения.
# Важным фактором при выборе метода является распределение признаков с пропусками.
# Давайте выведем их на экран. 
# pandas это можно сделать с помощью метода hist():
#cols = cols_with_null.index
#sber_data[cols].hist(figsize=(20, 8));


#Заполнение значений осуществляется с помощью метода fillna().
# Главный параметр метода — value (значение, на которое происходит заполнение данных в столбце).
# Если метод вызывается от имени всего DataFrame, то в качестве value можно использовать словарь,
# где ключи — названия столбцов таблицы, а значения словаря — заполняющие константы. 
#Создадим такой словарь, соблюдая рекомендации, приведённые выше, а также копию исходной таблицы.
# Произведём операцию заполнения с помощью метода fillna() и удостоверимся, что пропусков в данных больше нет:
#создаем копию исходной таблицы
fill_data = sber_data.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'life_sq': fill_data['full_sq'],
    'metro_min_walk': fill_data['metro_min_walk'].median(),
    'metro_km_walk': fill_data['metro_km_walk'].median(),
    'railroad_station_walk_km': fill_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': fill_data['railroad_station_walk_min'].median(),
    'hospital_beds_raion': fill_data['hospital_beds_raion'].mode()[0],
    'preschool_quota': fill_data['preschool_quota'].mode()[0],
    'school_quota': fill_data['school_quota'].mode()[0],
    'floor': fill_data['floor'].mode()[0]
}
#заполняем пропуски в соответствии с заявленным словарем
fill_data = fill_data.fillna(values)
#выводим результирующую долю пропусков
fill_data.isnull().mean()


#Посмотрим, на то, как изменились распределения наших признаков:
#cols = cols_with_null.index
#fill_data[cols].hist(figsize=(20, 8));
#Обратите внимание на то, как сильно изменилось распределение для признака hospital_beds_raion.
# Это связано с тем, что мы заполнили модальным значением почти 47 % общих данных.
# В результате мы кардинально исказили исходное распределение признака, что может плохо сказаться на модели.




#Заполнение недостающих значений константами с добавлением индикатора
# Посмотрим на реализацию. Как обычно, создадим копию indicator_data исходной таблицы.
# В цикле пройдёмся по столбцам с пропусками и будем добавлять в таблицу новый признак (с припиской "was_null"), который получается из исходного с помощью применения метода isnull().
# После чего произведём обычное заполнение пропусков, которое мы совершали ранее, и выведем на экран число отсутствующих значений в столбце, чтобы убедиться в результате:
#создаем копию исходной таблицы
indicator_data = sber_data.copy()
#в цикле пробегаемся по названиям столбцов с пропусками
for col in cols_with_null.index:
    #создаем новый признак-индикатор как col_was_null
    indicator_data[col + '_was_null'] = indicator_data[col].isnull()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски   
values = {
    'life_sq': indicator_data['full_sq'],
    'metro_min_walk': indicator_data['metro_min_walk'].median(),
    'metro_km_walk': indicator_data['metro_km_walk'].median(),
    'railroad_station_walk_km': indicator_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': indicator_data['railroad_station_walk_min'].median(),
    'hospital_beds_raion': indicator_data['hospital_beds_raion'].mode()[0],
    'preschool_quota': indicator_data['preschool_quota'].mode()[0],
    'school_quota': indicator_data['school_quota'].mode()[0],
    'floor': indicator_data['floor'].mode()[0]
}
#заполняем пропуски в соответствии с заявленным словарем
indicator_data = indicator_data.fillna(values)
#выводим результирующую долю пропусков
#indicator_data.isnull().mean()
#display(indicator_data.head())
#Метод исходит из предположения, что, если дать модели информацию о том, что в ячейке ранее была пустота, 
# то она будет меньше доверять таким записям и меньше учитывать её в процессе обучения. Иногда такие фишки действительно работают, 
# иногда не дают эффекта, а иногда и вовсе могут ухудшить результат обучения и затруднить процесс обучения.


#Комбинирование методов

#Наверняка вы уже догадались, что необязательно использовать один метод. Вы можете их комбинировать. Например, мы можем:
#удалить столбцы, в которых более 30 % пропусков;
#удалить записи, в которых более двух пропусков одновременно;
#заполнить оставшиеся ячейки константами.
#Посмотрим на реализацию такого подхода в коде:
#создаём копию исходной таблицы
combine_data = sber_data.copy()

#отбрасываем столбцы с числом пропусков более 30% (100-70)
n = combine_data.shape[0] #число строк в таблице
thresh = n*0.7
combine_data = combine_data.dropna(thresh=thresh, axis=1)

#отбрасываем строки с числом пропусков более 2 в строке
m = combine_data.shape[1] #число признаков после удаления столбцов
combine_data = combine_data.dropna(thresh=m-2, axis=0)

#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'life_sq': combine_data['full_sq'],
    'metro_min_walk': combine_data['metro_min_walk'].median(),
    'metro_km_walk': combine_data['metro_km_walk'].median(),
    'railroad_station_walk_km': combine_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': combine_data['railroad_station_walk_min'].median(),
    'preschool_quota': combine_data['preschool_quota'].mode()[0],
    'school_quota': combine_data['school_quota'].mode()[0],
    'floor': combine_data['floor'].mode()[0]
}
#заполняем оставшиеся записи константами в соответствии со словарем values
combine_data = combine_data.fillna(values)
#выводим результирующую долю пропусков
#display(combine_data.isnull().mean())

#print(combine_data.shape)

In [ ]:
#Пусть у нас есть признак, по которому мы будем искать выбросы.
# Давайте рассчитаем его статистические показатели (минимум, максимум, среднее, квантили)
# и по ним попробуем определить наличие аномалий.
sber_data['life_sq'].describe()

#Найдём число квартир с нулевой жилой площадью:
#print(sber_data[sber_data['life_sq'] == 0].shape[0])

#А теперь выведем здания с жилой площадью более 7 000 квадратных метров:
#display(sber_data[sber_data['life_sq'] > 7000])


#Логичен вопрос: а много ли у нас таких квартир, у которых жилая площадь больше, чем суммарная?
outliers = sber_data[sber_data['life_sq'] > sber_data['full_sq']]
#print(outliers.shape[0])


#Таких квартир оказывается 37 штук. Подобные наблюдения уже не поддаются здравому смыслу — они являются ошибочными, 
# и от них стоит избавиться. Для этого можно воспользоваться методом drop() и удалить записи по их индексам:
cleaned = sber_data.drop(outliers.index, axis=0)
#print(f'Результирующее число записей: {cleaned.shape[0]}')

#Ещё пример: давайте посмотрим на признак числа этажей (floor).
#display(sber_data['floor'].describe())

#Снова видим подозрительную максимальную отметку в 77 этажей. Проверим все квартиры, которые находятся выше 50 этажей:
#display(sber_data[sber_data['floor']> 50])
#Всего одна квартира в Ломоносовском районе. Пора идти в интернет в поиске самых высоких зданий в Москве! 

#На гистограмме мы можем увидеть потенциальные выбросы как низкие далеко отстоящие от основной группы столбцов «пеньки», 
# а на коробчатой диаграмме — точки за пределами усов

#Построим гистограмму и коробчатую диаграмму для признака полной площади (full_sq):
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
#histplot = sns.histplot(data=sber_data, x='full_sq', ax=axes[0]);
#histplot.set_title('Full Square Distribution');
#boxplot = sns.boxplot(data=sber_data, x='full_sq', ax=axes[1]);
#boxplot.set_title('Full Square Boxplot');



#В соответствии с этим алгоритмом напишем функцию outliers_iqr(), которая вам может ещё не раз пригодиться в реальных задачах.
# Эта функция принимает на вход DataFrame и признак, по которому ищутся выбросы, а затем возвращает потенциальные выбросы,
# найденные с помощью метода Тьюки, и очищенный от них датасет.

#Квантили вычисляются с помощью метода quantile().
# Потенциальные выбросы определяются при помощи фильтрации данных по условию выхода за пределы верхней или нижней границы.

#Одним из таких подходов является метод межквартильного размаха (его еще называют методом Тьюки), 
# который используется для построения коробчатой диаграммы.

#вычислить 25-ый и 75-ый квантили (первый и третий квартили) — и  для признака, который мы исследуем;

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

#Применим эту функцию к таблице sber_data и признаку full_sq, а также выведем размерности результатов:
outliers, cleaned = outliers_iqr(sber_data, 'full_sq')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')
#display(outliers.head())
#display(cleaned.head())


Число выбросов по методу Тьюки: 963
Результирующее число записей: 29508


,id,full_sq,life_sq,floor,sub_area,preschool_quota,preschool_education_centers_raion,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,...,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,theater_km,museum_km,ecology,mosque_count_1000,price_doc
79,80,133,64.0,2.0,Izmajlovo,1313.0,4,4339.0,6,0,...,0.804087,0.435858,0.676590,2.098633,0.968482,7.141546,5.413698,good,0,17600000
128,129,325,325.0,7.0,Ivanovskoe,2697.0,7,9439.0,8,1,...,3.310054,0.602138,0.369984,1.104165,1.247850,2.194396,1.143931,poor,0,5000000
146,147,102,64.0,11.0,Sokolinaja Gora,643.0,4,5180.0,4,0,...,0.999584,1.426903,1.163062,1.627478,0.601487,3.449316,1.557170,excellent,0,4600000
147,148,117,108.0,20.0,Nagatino-Sadovniki,2508.0,4,5254.0,6,0,...,0.237372,0.437805,0.157045,1.855743,1.549737,4.452140,2.014427,excellent,0,14103600
170,171,115,60.0,7.0,Krylatskoe,3092.0,7,7478.0,7,0,...,0.727716,0.681249,0.165629,0.287008,1.147352,12.239214,7.247341,good,0,37000000


,id,full_sq,life_sq,floor,sub_area,preschool_quota,preschool_education_centers_raion,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,...,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,theater_km,museum_km,ecology,mosque_count_1000,price_doc
0,1,43,27.0,4.0,Bibirevo,5001.0,5,11065.0,5,0,...,0.637189,0.947962,0.177975,0.625783,0.628187,14.053047,7.389498,good,0,5850000
1,2,34,19.0,3.0,Nagatinskij Zaton,3119.0,5,6237.0,8,0,...,0.688796,1.072315,0.273345,0.967821,0.471447,6.829889,0.709260,excellent,0,6000000
2,3,43,29.0,2.0,Tekstil'shhiki,1463.0,4,5580.0,7,0,...,1.543049,0.391957,0.158072,3.178751,0.755946,4.273200,3.156423,poor,0,5700000
3,4,89,50.0,9.0,Mitino,6839.0,9,17063.0,10,0,...,0.934273,0.892674,0.236455,1.031777,1.561505,16.990677,16.041521,good,0,13100000
4,5,77,77.0,4.0,Basmannoe,3240.0,7,7770.0,9,0,...,0.077901,0.810801,0.376838,0.378756,0.121681,1.112486,1.800125,excellent,0,16331452
